In [4]:
import networkx as nx
from ge.models.deepwalk import DeepWalk
from ge.models.node2vec import Node2Vec
from ge.models.line import LINE
from ge.models.struc2vec import Struc2Vec
from ge.models.sdne import SDNE
from stellargraph import datasets
from IPython.display import display, HTML

%matplotlib inline

In [5]:
dataset = datasets.Cora()
display(HTML(dataset.description))
G, node_subjects = dataset.load(largest_connected_component_only=True)

In [6]:
G = nx.graph(G)

TypeError: 'module' object is not callable

In [7]:
%time
#G = nx.read_edgelist('karate-mirrored.txt')#'./data/wiki/Wiki_edgelist.txt',create_using=nx.DiGraph(),nodetype=None,data=[('weight',int)])# Read graph
G =  nx.read_edgelist('./cora/cora.cites')

model = DeepWalk(G,walk_length=10,num_walks=80,workers=1)#init model
model.train(window_size=5)# train model
embeddings = model.get_embeddings()# get embedding vectors

Wall time: 0 ns


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.3s finished


Learning embedding vectors...
Learning embedding vectors done!


In [12]:
#G=nx.read_edgelist('./data/wiki/Wiki_edgelist.txt',create_using = nx.DiGraph(), nodetype = None, data = [('weight', int)])#read graph
G = nx.read_edgelist('karate-mirrored.txt')#'./data/wiki/Wiki_edgelist.txt',create_using=nx.DiGraph(),nodetype=None,data=[('weight',int)])# Read graph

model = Node2Vec(G, walk_length = 10, num_walks = 80,p = 0.25, q = 4, workers = 1)#init model
model.train(window_size = 5, itr = 3)# train model
embeddings = model.get_embeddings()# get embedding vectors

Preprocess transition probs...
Learning embedding vectors...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


Learning embedding vectors done!


In [3]:
#G=nx.read_edgelist('./data/wiki/Wiki_edgelist.txt',create_using = nx.DiGraph(), nodetype = None, data = [('weight', int)])#read graph
G = nx.read_edgelist('karate-mirrored.txt')#'./data/wiki/Wiki_edgelist.txt',create_using=nx.DiGraph(),nodetype=None,data=[('weight',int)])# Read graph

model = model = Struc2Vec(G, 10, 80, workers=4, verbose=40, ) #init model
model.train(window_size = 5)#, itr = 3)# train model
embeddings = model.get_embeddings()# get embedding vectors

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.8s remaining:    0.8s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.8s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.7s remaining:    0.7s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.0s finished


Learning representation...
Learning representation done!


In [9]:
G = nx.read_edgelist('karate-mirrored.txt')#'./d

model = SDNE(G,hidden_size=[256,128]) #init model
model.train(batch_size=3000,epochs=40,verbose=2)# train model
embeddings = model.get_embeddings()# get embedding vectors

batch_size(3000) > node_size(68),set batch_size = 68
Epoch 1/40


NotImplementedError: in user code:

    C:\Users\tarik\Anaconda3\envs\graphs\lib\site-packages\tensorflow\python\keras\engine\training.py:855 train_function  *
        return step_function(self, iterator)
    C:\Users\tarik\source\repos\COVIDGraphProject\src\GraphEmbedding\ge\models\sdne.py:36 loss_2nd  *
        b_ = np.ones_like(y_true)
    <__array_function__ internals>:6 ones_like  **
        
    C:\Users\tarik\Anaconda3\envs\graphs\lib\site-packages\numpy\core\numeric.py:263 ones_like
        res = empty_like(a, dtype=dtype, order=order, subok=subok, shape=shape)
    <__array_function__ internals>:6 empty_like
        
    C:\Users\tarik\Anaconda3\envs\graphs\lib\site-packages\tensorflow\python\framework\ops.py:870 __array__
        " a NumPy call, which is not supported".format(self.name))

    NotImplementedError: Cannot convert a symbolic Tensor (IteratorGetNext:2) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not suppo

In [8]:
from ge.classify import read_node_label, Classifier
import numpy as np
from ge import Node2Vec
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
import networkx as nx
from sklearn.manifold import TSNE

In [10]:
def plot_embeddings(embeddings,):
    X, Y = read_node_label('karate-mirrored.txt')
    c = [plt.cm.tab20(i) for i in range(20)]
    for i in range(20):
        c.append(plt.cm.tab20b(i))
        c.append(plt.cm.tab20c(i) )
    for i in range(10):
        c.append(plt.cm.tab10(i))
        c.append(plt.cm.Paired(i))
        c.append(plt.cm.Set3(i))
    #print(len(c))
    colors = iter([i for i in c])
    emb_list = []
    for k in X:
        emb_list.append(embeddings[k])
    #print(X)
    emb_list = np.array(emb_list)
    
    model = TSNE(n_components=2)
    node_pos = model.fit_transform(emb_list)

    color_idx = {}
    for i in range(len(X)):
        color_idx.setdefault(Y[i][0], [])
        color_idx[Y[i][0]].append(i)
    plt.figure(figsize=(8, 8), dpi=80)
    for c, idx in color_idx.items():
        C = next(colors)
        plt.scatter(node_pos[idx, 0], node_pos[idx, 1], label=c, c=C)
        #plt.annotate(c, xy=(node_pos[idx, 0], node_pos[idx, 1]),alpha=.5)
    plt.legend( bbox_to_anchor=(0.6,0.5),ncol=4)
    plt.savefig('cora.png')
    plt.show()

In [11]:
plot_embeddings(embeddings)

KeyError: '1'